In [9]:
# 传统神经网络存在的问题:
#     权值太多,计算量太大
#     权值太多,需要大量样本进行训练(过拟合或欠拟合) 
# NN:  fully connected neural net
# CNN: locally connected neural net

In [10]:
# CNN:
#     池化 max-pooling  mean-pooling random-pooling
#     卷积操作: 
#         same padding 
#         可能给平面外部补0 卷积窗口采样后得到一个跟原本平面大小相同的平面
#         valid padding
#         不会超出平面外部 卷积窗口采样后得到一个比原来平面小的平面


In [15]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [16]:
mnist=input_data.read_data_sets("MNIST_data",one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [ ]:
#每个批次的大小
batch_size=100
#计算一共多少批次
n_batch=mnist.train.num_examples//batch_size

def weight_variable(shape):
    initial=tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial=tf.constant(0.1,shape=shape)
    return tf.Variable(initial)

#卷积层
def conv2d(x,W):
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')
#池化层
def max_pool_2x2(x):
#     strides=[1,_,1,_] ksize=[1,_,_1]
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

x=tf.placeholder(tf.float32,[None,784])
y=tf.placeholder(tf.float32,[None,10])

#改变x的格式转为4D向量[batch,in_height,in_width,in_channel]
x_image=tf.reshape(x,[-1,28,28,1])

#初始化第一个卷积层的权值和偏置
W_conv1=weight_variable([5,5,1,32])#5x5的采样窗口,32个卷积核从1个平面抽取特征
b_conv1=bias_variable([32])
#把x_image和权值向量进行卷积,再加上偏置值,然后应用于relu激活函数
h_conv1=tf.nn.relu(conv2d(x_image,W_conv1)+b_conv1)
h_pool1=max_pool_2x2(h_conv1)

#初始化第二个卷积层
W_conv2=weight_variable([5,5,32,64])#5x5的采样窗口,32个卷积核从1个平面抽取特征
b_conv2=bias_variable([64])
#把h_pool1和权值向量进行卷积,再加上偏置值,然后应用于relu激活函数
h_conv2=tf.nn.relu(conv2d(h_pool1,W_conv2)+b_conv2)
h_pool2=max_pool_2x2(h_conv2)

# 28x28的图片第一次卷积后还是28x28,第一次池化变为14x14
# 第二次卷积后14x14,第二次池化后为7x7
# 进行上面操作后得到64张7x7的平面

#初始化第一个全连接层的权值
W_fcl=weight_variable([7*7*64,1024])#上一层7x7x64个神经元,全连接层有1024个神经元
b_fcl=bias_variable([1024])

#把池化层2的输出扁平化为一维
h_pool2_flat=tf.reshape(h_pool2,[-1,7*7*64])
#求第一个全连接层的输出
h_fcl=tf.nn.relu(tf.matmul(h_pool2_flat,W_fcl)+b_fcl)

#keep_prob用来表示神经元的输出概率
keep_prob=tf.placeholder(tf.float32)
h_fcl_drop=tf.nn.dropout(h_fcl,keep_prob)

#初始化第二个全连接层
W_fc2=weight_variable([1024,10])
b_fc2=bias_variable([10])
#计算输出
prediction=tf.nn.softmax(tf.matmul(h_fcl_drop,W_fc2)+b_fc2)

#交叉熵代价函数
cross_entropy=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prediction))
#使用AdamOptimizer进行优化
train_step=tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
#结果存放在一个布尔列表中
correct_prediction=tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))
#准确率
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(21):
        for batch in range(n_batch):
            batch_xs,batch_ys=mnist.train.next_batch(batch_size)
            sess.run(train_step,feed_dict={x:batch_xs,y:batch_ys,keep_prob:0.7})
        
        acc=sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels,keep_prob:1.0})
        print("Iteration "+str(epoch)+", Testing accuracy "+str(acc))





Iteration 0, Testing accuracy 0.9545
